In [1]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [2]:
english_file = 'english.txt'
kannada_file = 'kannada.txt'

# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

kannada_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ', 
                      'ँ', 'ఆ', 'ఇ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 
                      'ಅ', 'ಆ', 'ಇ', 'ಈ', 'ಉ', 'ಊ', 'ಋ', 'ೠ', 'ಌ', 'ಎ', 'ಏ', 'ಐ', 'ಒ', 'ಓ', 'ಔ', 
                      'ಕ', 'ಖ', 'ಗ', 'ಘ', 'ಙ', 
                      'ಚ', 'ಛ', 'ಜ', 'ಝ', 'ಞ', 
                      'ಟ', 'ಠ', 'ಡ', 'ಢ', 'ಣ', 
                      'ತ', 'ಥ', 'ದ', 'ಧ', 'ನ', 
                      'ಪ', 'ಫ', 'ಬ', 'ಭ', 'ಮ', 
                      'ಯ', 'ರ', 'ಱ', 'ಲ', 'ಳ', 'ವ', 'ಶ', 'ಷ', 'ಸ', 'ಹ', 
                      '಼', 'ಽ', 'ಾ', 'ಿ', 'ೀ', 'ು', 'ೂ', 'ೃ', 'ೄ', 'ೆ', 'ೇ', 'ೈ', 'ೊ', 'ೋ', 'ೌ', '್', 'ೕ', 'ೖ', 'ೞ', 'ೣ', 'ಂ', 'ಃ', 
                      '೦', '೧', '೨', '೩', '೪', '೫', '೬', '೭', '೮', '೯', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [3]:
index_to_kannada = {k:v for k,v in enumerate(kannada_vocabulary)}
kannada_to_index = {v:k for k,v in enumerate(kannada_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [4]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(kannada_file, 'r') as file:
    kannada_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
kannada_sentences = [sentence.rstrip('\n') for sentence in kannada_sentences]

In [5]:
english_sentences[:15]

['hes a scientist.',
 "'but we speak the truth aur ye sach hai ke gujarat mein vikas pagal hogaya hai,'' rahul gandhi further said in banaskantha",
 '8 lakh crore have been looted.',
 'i read a lot into this as well.',
 "she was found dead with the phone's battery exploded close to her head the following morning.",
 'how did mankind come under satans rival sovereignty?',
 'and then i became prime minister.',
 'what about corruption?',
 'no differences',
 '"""the shooting of the film is 90 percent done."',
 'the special statute',
 '"then the king said to ittai the gittite, ""why do you also go with us? return, and stay with the king. for you are a foreigner, and also an exile. return to your own place."',
 'what happened at the un general assembly?',
 '720p hd recording',
 'the meeting was attended by prime minister narendra modi, home minister amit shah and defence minister rajnath singh, among others.']

In [6]:
kannada_sentences[:15]

['ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.',
 '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"',
 'ಕಳ್ಳತನವಾಗಿದ್ದ 8 ಲಕ್ಷ ರೂ.',
 'ಇದರ ಬಗ್ಗೆ ನಾನೂ ಸಾಕಷ್ಟು ಓದಿದ್ದೇನೆ.',
 'ಆಕೆಯ ತಲೆಯ ಹತ್ತಿರ ಇರಿಸಿಕೊಂಡಿದ್ದ ಫೋನ್\u200cನ ಬ್ಯಾಟರಿ ಸ್ಫೋಟಗೊಂಡು ಆಕೆ ಮೃತಪಟ್ಟಿದ್ದಾಳೆ ಎನ್ನಲಾಗಿದೆ.',
 'ಮಾನವಕುಲವು ಸೈತಾನನ ಆಳಿಕೆಯ ಕೆಳಗೆ ಬಂದದ್ದು ಹೇಗೆ?',
 'ನಂತರ ಪ್ರಧಾನಿ ಕೂಡ ಆಗುತ್ತೇನೆ.',
 'ಭ್ರಷ್ಟಾಚಾರ ಏಕಿದೆ?',
 '‘ಅನುಪಾತದಲ್ಲಿ ವ್ಯತ್ಯಾಸವಿಲ್ಲ’',
 'ಆ ಚಿತ್ರದ ಶೇ 90ರಷ್ಟು ಚಿತ್ರೀಕರಣವೂ ಈಗಾಗಲೇ ಮುಗಿದು ಹೋಗಿದೆ.',
 'ವಿಶೇಷ ಕಾನೂನು',
 'ಆಗ ಅರಸನು ಗಿತ್ತೀಯನಾದ ಇತ್ತೈಯನ್ನು ನೋಡಿ--ನೀನು ನಮ್ಮ ಸಂಗಡ ಬರುವದು ಯಾಕೆ? ನಿನ್ನ ಸ್ಥಳಕ್ಕೆ ಹಿಂದಿರುಗಿ ಹೋಗಿ ಅರಸನ ಸಂಗಡ ಇರು. ಯಾಕಂದರೆ ನೀನು ಸೆರೆಹಿಡಿಯಲ್ಪಟ್ಟವನಾದ ಅನ್ಯದೇಶದವನು.',
 'ವಿಶ್ವ ಗೋ ಸಮ್ಮೇಳನದ ಅಂಗಳದಲ್ಲಿ ಏನೇನು ನಡೆದಿದೆ?',
 '720p ಹೈ ಡೆಫಿನಿಷನ್ ವಿಡಿಯೋ ರೆಕಾರ್ಡಿಂಗ್',
 'ಪ್ರಧಾನ ಮಂತ್ರಿ ನರೇಂದ್ರ ಮೋದಿ, ರಕ್ಷಣಾ ಸಚಿವ ರಾಜನಾಥ್ ಸಿಂಗ್ ಮತ್ತು ಕೇಂದ್ರ ಗೃಹ ಸಚಿವ ಅಮಿತ್ ಷಾ ಅವರು ಮಸೂದೆಯ ಬಗ್ಗೆ ಸಾರ್ವಜನಿಕ ಚರ್ಚೆ ಗೆ ಬರುವಂತೆ ಸಂಘ ಸವಾಲು ಹಾಕಿದೆ.']

In [7]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in kannada_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 172.0
97th percentile length English: 178.0


In [8]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = kannada_sentences[index], english_sentences[index]
    if is_valid_length(kannada_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(kannada_sentence, kannada_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 164022


In [9]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [10]:
kannada_sentences[:15]

['ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.',
 '"ಆದರೆ ಸತ್ಯ ಹೊರ ಬಂದೇ ಬರುತ್ತದೆ ಎಂದು ಹೇಳಿದ ರಾಹುಲ್ ಗಾಂಧಿ, ""ಸೂರತ್ ಜನರು ಚೀನಾದ ಜತೆ ಸ್ಪರ್ಧೆ ನಡೆಸುತ್ತಿದ್ದಾರೆ"',
 'ಕಳ್ಳತನವಾಗಿದ್ದ 8 ಲಕ್ಷ ರೂ.',
 'ಇದರ ಬಗ್ಗೆ ನಾನೂ ಸಾಕಷ್ಟು ಓದಿದ್ದೇನೆ.',
 'ಮಾನವಕುಲವು ಸೈತಾನನ ಆಳಿಕೆಯ ಕೆಳಗೆ ಬಂದದ್ದು ಹೇಗೆ?',
 'ನಂತರ ಪ್ರಧಾನಿ ಕೂಡ ಆಗುತ್ತೇನೆ.',
 'ಭ್ರಷ್ಟಾಚಾರ ಏಕಿದೆ?',
 'ಆ ಚಿತ್ರದ ಶೇ 90ರಷ್ಟು ಚಿತ್ರೀಕರಣವೂ ಈಗಾಗಲೇ ಮುಗಿದು ಹೋಗಿದೆ.',
 'ವಿಶೇಷ ಕಾನೂನು',
 'ಆಗ ಅರಸನು ಗಿತ್ತೀಯನಾದ ಇತ್ತೈಯನ್ನು ನೋಡಿ--ನೀನು ನಮ್ಮ ಸಂಗಡ ಬರುವದು ಯಾಕೆ? ನಿನ್ನ ಸ್ಥಳಕ್ಕೆ ಹಿಂದಿರುಗಿ ಹೋಗಿ ಅರಸನ ಸಂಗಡ ಇರು. ಯಾಕಂದರೆ ನೀನು ಸೆರೆಹಿಡಿಯಲ್ಪಟ್ಟವನಾದ ಅನ್ಯದೇಶದವನು.',
 'ವಿಶ್ವ ಗೋ ಸಮ್ಮೇಳನದ ಅಂಗಳದಲ್ಲಿ ಏನೇನು ನಡೆದಿದೆ?',
 'ಪ್ರಧಾನ ಮಂತ್ರಿ ನರೇಂದ್ರ ಮೋದಿ, ರಕ್ಷಣಾ ಸಚಿವ ರಾಜನಾಥ್ ಸಿಂಗ್ ಮತ್ತು ಕೇಂದ್ರ ಗೃಹ ಸಚಿವ ಅಮಿತ್ ಷಾ ಅವರು ಮಸೂದೆಯ ಬಗ್ಗೆ ಸಾರ್ವಜನಿಕ ಚರ್ಚೆ ಗೆ ಬರುವಂತೆ ಸಂಘ ಸವಾಲು ಹಾಕಿದೆ.',
 'ಎಂಬುದು ಬಹಳ ದೀರ್ಘ ಕಾಲದಿಂದಲೂ ಚರ್ಚಿತವಾಗುತ್ತಿರುವ ವಿಷಯ.',
 'ಇಲ್ಲಿಗೆ ಬರಲು ಬಸ್ ಸೌಕರ್ಯವೂ ಇಲ್ಲ.',
 'ಆ ಪರಂಪರೆ ಯಾಕೆ ಮುನ್ನೆಲೆಗೆ ಬರಲಿಲ್ಲ?']

In [11]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 3
max_sequence_length = 200
kn_vocab_size = len(kannada_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          kannada_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [12]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (attention): MultiHeadAt

In [13]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [14]:
dataset = TextDataset(english_sentences, kannada_sentences)

In [15]:
len(dataset)

164022

In [16]:
dataset[12]

('it has been under discussion for a long time.',
 'ಎಂಬುದು ಬಹಳ ದೀರ್ಘ ಕಾಲದಿಂದಲೂ ಚರ್ಚಿತವಾಗುತ್ತಿರುವ ವಿಷಯ.')

In [17]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [18]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('hes a scientist.', "'but we speak the truth aur ye sach hai ke gujarat mein vikas pagal hogaya hai,'' rahul gandhi further said in banaskantha", '8 lakh crore have been looted.', 'i read a lot into this as well.', 'how did mankind come under satans rival sovereignty?', 'and then i became prime minister.', 'what about corruption?', '"""the shooting of the film is 90 percent done."', 'the special statute', '"then the king said to ittai the gittite, ""why do you also go with us? return, and stay with the king. for you are a foreigner, and also an exile. return to your own place."', 'what happened at the un general assembly?', 'the meeting was attended by prime minister narendra modi, home minister amit shah and defence minister rajnath singh, among others.', 'it has been under discussion for a long time.', 'buses cannot get there.', 'why then this tradition was not thought of?', 'kashmiri youth join indian army', 'basic amenities elude this village', 'off-budget borrowings of the state

In [19]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=kannada_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [20]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [22]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 9

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Kannada Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == kannada_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_kannada[idx.item()]
            print(f"Kannada Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("i love you mom",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (i love you mom) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 2.067136764526367
English: hes a scientist.
Kannada Translation: ಇವರು ಸಂಶೋಧಕ ಸ್ವಭಾವದವರು.
Kannada Prediction: ಅದರು ಮ್ಭ್ಧಿ ಮಂಥಾಾನಿ ುು.
Evaluation translation (i love you mom) : ('ಅದಕ್ಕೆ ಸಂಪೂರ್ಣ ಸ್ಥಳ್ಳಿ<END>',)
-------------------------------------------
Iteration 100 : 1.9166909456253052
English: she ate it.
Kannada Translation: ಅವಳು ಅವನಿಗೆ ಊಟ ಹಾಕಿದಳೂ.
Kannada Prediction: ಅದರು ನವರ್ಗೆ ಹರ್ನೇಗೆತ್ು 
Evaluation translation (i love you mom) : ('ಅವರು ಸಂಪೂರ್ಣ ಮಾಡಿದ್ದಾರೆ.<END>',)
-------------------------------------------
Iteration 200 : 1.925971508026123
English: caste and religion were unknown.
Kannada Translation: ಜಾತಿ, ಬೇಧ ಎಂಬುದೇ ಗೊತ್ತಿರಲಿಲ್ಲ.
Kannada Prediction: ಇನರ್  ಕಿಕಾಕಂದುದು ಎುತ್ತಿದುಿಲ್ಲ.
Evaluation translation (i love you mom) : ('ಇದು ನಿಮ್ಮ ಸಂಗ್ರೆಸ್ ಹೇಳಿದ್ದಾರೆ.<END>',)
-------------------------------------------
Iteration 300 : 1.9987924098968506
English: seeing this, ruler was elated and told his son that the strength of the rabbit is due to the valour of the reg

## Testing 

In [23]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_kannada[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [28]:
translation = translate("what should we do when the day starts?")
print(translation)


ಅವರು ಏನು ಮಾಡಬೇಕು?<END>


In [25]:
translation = translate("i love you amma")
print(translation)

ನಾನು ಮಾತನಾಡುತ್ತೇನೆ<END>


In [27]:
translation = translate("i love you dad")
print(translation)

ನಾನು ಕೊಟ್ಟಿದ್ದೇನೆ ಎಂದು ನಾನು ಹೇಳುತ್ತೇನೆ<END>


In [29]:
translation = translate("my name is ranjith chodavarapu")
print(translation)

ನನ್ನ ಮುಖ್ಯಮಂತ್ರಿ ಅಮಿತ್ ಶಾ ಅವರ ಪ್ರಮುಖ ಪಾತ್ರ<END>


In [30]:
translation = translate("my favorite food is curd rice ")
print(translation)

ನಾನು ಮಾಡಿದ ಮುಖ್ಯಮಂತ್ರಿ ಅವರ ಪತ್ನಿ<END>


In [31]:
translation = translate("what's your name ")
print(translation)

ನಿಮ್ಮ ಮನೆಯಲ್ಲಿ ಏನು?<END>


In [32]:
translation = translate("project is done")
print(translation)

ಪ್ರತಿಭಟನೆ ನಡೆಸಿದ ಪ್ರತಿಭಟನೆ<END>


In [33]:
translation = translate("thank you")
print(translation)

ನಿಮ್ಮ ಗೊತ್ತಿಲ್ಲ.<END>


In [34]:
translation = translate("had your lunch")
print(translation)

ನಿಮ್ಮ ಮಕ್ಕಳನ್ನು ನೀಡಿದ್ದಾರೆ<END>


In [36]:
translation = translate("good morning")
print(translation)

ಹೆಚ್ಚು ಮಾಡುವುದು<END>


In [37]:
translation = translate("good afternoon")
print(translation)

ಕಾರ್ಯಕ್ರಮದಲ್ಲಿ ಕಾರ್ಯಕ್ರಮ<END>


In [38]:
translation = translate("good night")
print(translation)

ದೇವರ ಹೇಳುತ್ತಿದ್ದಾರೆ.<END>
